# Reporte Práctica 2, Parte 2
*Lau, Santi, Rafa y Sebas*
**Optimización Avanzada**
***

## Introducción

 En esta última parte de las prácticas desarrolladas durante el curso se reimplementan los métodos que se han desarrollado anteriormente, haciendo el código más eficiente mediante el uso de *multiprocessing*.

## Definición del Problema

 En este ocasión trabajaremos nuevamente con el dataset que contiene las ciudades de China, `ch71009.tsp` que se encuentra en el directorio `datasets` de este repositorio$^{[2]}$. Tomaremos una muestra aleatoria de $100$ ciudades y $1000$ hormigas, de esta manera podremos comprobar que se ha hecho una mejora en el tiempo de cómputo en el algoritmo.

## Perfilamiento

 La reimplementación desarrollada se hizo sobre la clase `colony` a una versión que utiliza multiprocesamiento, que se apoya de la librería `multiprocessing` que revisamos en el libro del curso $^{[1]}$ sobre el cómputo en paralelo.

 De los métodos que se desarrollaron anteriormente, notemos que el método `solve_tsp` es el que toma más tiempo durante su ejecución, por lo que en esta práctica se implementó la siguiente solución:
 
 - Se envía un número determinado de hormigas a un *pool* de workers, donde para cada iteración, las hormigas van a recorrer el grafo para buscar una solución.
 - En cada iteración se actualiza el número de feromonas del grafo según los recorridos determinados por cada una de las hormigas del punto anterior.

## Implementación con `ant_colony`

In [20]:
#!pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-1-segunda-parte-ltejadal.git#egg=ant-colony&subdirectory=src" &> /dev/null

In [26]:
# librerias
import ant_colony as ac
import time

In [41]:
path_china = 'datasets/ch71009.tsp'
G = ac.read_coord_data(path_china, n_cities=100, seed=1999)

Problem with 71009 cities. Selected 100.


 Primero echemos un vistazo a las características de la máquina en la que se ejecuta este reporte:

In [42]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:            7
CPU MHz:             3099.932
BogoMIPS:            4999.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand h

In [43]:
n_ants = 1000

Vamos a medir el tiempo de ejecución con la clase anterior y compararla contra la nueva.

In [44]:
colony_old = ac.colony(G, init_node=0,  n_ants=n_ants)

In [45]:
start_time = time.time()
colony_old.solve_tsp()
end_time = time.time()

In [46]:
secs = end_time-start_time
print("La solucion sin pool tomó", secs, "segundos." )
print(f"Distancia {colony_old.best_dist} kms.")

La solucion sin pool tomó 2699.3665103912354 segundos.
Distancia 1119.0648040721273 kms.


## Implementación con `colony_mutiw`

In [47]:
colony_mw = ac.colony_multiw(G, init_node=0,  n_ants= n_ants, n_workers=4)

In [48]:
start_time = time.time()
colony_mw.solve_tsp()
end_time = time.time()

In [49]:
secs = end_time-start_time
print("La solucion con pool de workers tomó", secs, "segundos." )
print(f"Distancia {colony_mw.best_dist} kms.")

La solucion con pool de workers tomó 452.3769097328186 segundos.
Distancia 1119.0648040721273 kms.


El tiempo de ejecución se ha reducido casi 6 veces respecto a la clase que habíamos definido anteriormente.

## Conclusión

 Notemos que en ambos casos la distancia se mantiene fija, pero el tiempo de ejecución se ha reducido sustancialmente. Con esta nueva implementación en la que se utiliza cómputo en paralelo el tiempo de cómputo se redujo aproximadamente 6 veces, consiguiendo el objetivo de optimización del algoritmo.

## Referencias

1. [Cómputo en paralelo usando CPUs en un sistema de memoria compartida (SMC)](https://itam-ds.github.io/analisis-numerico-computo-cientifico/V.optimizacion_de_codigo/5.4/Computo_en_paralelo_usando_CPUS_en_SMC.html#multiprocessing) (2021) Erick Palacios
2. [National Travelling Salesman Problems](https://www.math.uwaterloo.ca/tsp/world/countries.html) (2017) University of Waterloo